# Import Dataset

In [69]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

from Preprocessing_functions import *

In [70]:
train_data = pd.read_csv('train_data.csv', index_col='Claim Identifier')
test_data = pd.read_csv('test_data.csv', index_col='Claim Identifier')

/var/folders/z6/_vn8cndd4x5dmx2cpjfdkjdr0000gn/T/ipykernel_26564/3470921380.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train_data.csv', index_col='Claim Identifier')


In [71]:
train_data.isna().sum()

Accident Date                          23134
Age at Injury                          19445
Alternative Dispute Resolution         19445
Assembly Date                              0
Attorney/Representative                19445
Average Weekly Wage                    48096
Birth Year                             48523
C-2 Date                               34005
C-3 Date                              406226
Carrier Name                           19445
Carrier Type                           19445
Claim Injury Type                      19445
County of Injury                       19445
COVID-19 Indicator                     19445
District Name                          19445
First Hearing Date                    442673
Gender                                 19445
IME-4 Count                           460668
Industry Code                          29403
Industry Code Description              29403
Medical Fee Region                     19445
OIICS Nature of Injury Description    593471
WCIO Cause

# Preprocessing

In [72]:
train_data = train_data[~(train_data.drop(columns=['Assembly Date']).isna().all(axis=1) & train_data['Assembly Date'].notna())]

In [73]:
X = train_data.drop(columns=['Claim Injury Type', 'WCB Decision', 'Agreement Reached'])
y = train_data['Claim Injury Type']

In [74]:
y.isna().sum()

np.int64(0)

In [75]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [76]:
def drop_description_columns(X_train, X_val):
    """
    Drop all columns in X_train and X_val that contain the word 'description' in their names (case-insensitive).
    """
    # Identify columns containing the word 'description'
    description_columns = X_train.columns[X_train.columns.str.contains('description', case=False, na=False)]
    
    # Drop these columns from both DataFrames
    X_train = X_train.drop(description_columns, axis=1)
    X_val = X_val.drop(description_columns, axis=1)
    
    return X_train, X_val

# Call the function
X_train, X_val = drop_description_columns(X_train, X_val)

In [77]:
CODE_COLUMNS = ['Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code']

DESCRIPTION_COLUMNS = ['WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description','Industry Code Description']

BOOLEAN_COLUMNS = ['Alternative Dispute Resolution', 'Attorney/Representative','COVID-19 Indicator']

date_order = ['Accident Date', 'C-2 Date','C-3 Date','Assembly Date', 'First Hearing Date']


In [78]:
numerical_columns = [
    'Average Weekly Wage']

In [79]:
col_minmax = ['Age at Injury', 'Birth Year', 'Number of Dependents']
col_standart = ['Accident Date','Assembly Date','C-2 Date','C-3 Date','Average Weekly Wage','First Hearing Date']

In [80]:
numerical_columns = [
    'Accident Date', 
    'Age at Injury', 
    'Assembly Date', 
    'Average Weekly Wage', 
    'Birth Year', 
    'C-2 Date', 
    'C-3 Date', 
    'First Hearing Date', 
    'IME-4 Count', 
    'Industry Code', 
    'WCIO Cause of Injury Code', 
    'WCIO Nature of Injury Code', 
    'WCIO Part Of Body Code', 
    'Number of Dependents',
]

categorical_features = ['Alternative Dispute Resolution',
 'Attorney/Representative',
 'Carrier Name',
 'Carrier Type',
 'County of Injury',
 'COVID-19 Indicator',
 'District Name',
 'Gender',
 'Industry Code',
 'Medical Fee Region',
 'WCIO Cause of Injury Code',
 'WCIO Nature of Injury Code',
 'WCIO Part Of Body Code',
 'Zip Code']


In [81]:
low_cardinality_cols = [col for col in categorical_features if X_train[col].nunique() < 10]
high_cardinality_cols = [col for col in categorical_features if X_train[col].nunique() > 10]

In [ ]:
def preprocessing_dum(X_train, X_val):
    convert_to_timestamp(X_train, X_val, date_order)
    convert_to_bool(X_train, X_val, col_names=BOOLEAN_COLUMNS)
    fill_missing_with_mode(X_train, X_val)
    feature_creation_has_Cdate (X_train, X_val)
    impute_mean_numerical(X_train, X_val, numerical_columns)
    code_columns_train = [col for col in X_train.columns if 'Code' in col]
    code_columns_val = [col for col in X_val.columns if 'Code' in col]

    X_train[code_columns_train] = X_train[code_columns_train].astype(str)
    X_val[code_columns_val] = X_val[code_columns_val].astype(str)
    scaling_minmax(X_train, X_val, col_minmax)
    scaling_standard(X_train, X_val, col_standart)
    
preprocessing_dum(X_train, X_val)

X_train.info()


/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions.py:301: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(mean_value, inplace=True)
/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions.py:302: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work 

<class 'pandas.core.frame.DataFrame'>
Index: 459220 entries, 5785935 to 6027959
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Accident Date                   459220 non-null  float64
 1   Age at Injury                   459220 non-null  float64
 2   Alternative Dispute Resolution  459220 non-null  object 
 3   Assembly Date                   459220 non-null  float64
 4   Attorney/Representative         459220 non-null  object 
 5   Average Weekly Wage             459220 non-null  float64
 6   Birth Year                      459220 non-null  float64
 7   C-2 Date                        459220 non-null  float64
 8   C-3 Date                        459220 non-null  float64
 9   Carrier Name                    459220 non-null  object 
 10  Carrier Type                    459220 non-null  object 
 11  County of Injury                459220 non-null  object 
 12  COVID-19 Indic

In [ ]:
X_train_encoded

In [ ]:

"""
def preprocessing_complex(X_train, X_val):
    convert_to_timestamp(X_train, X_val, date_order)
    convert_to_bool(X_train, X_val, col_names=BOOLEAN_COLUMNS)
    impute_mean_numerical(X_train, X_val, ['C-2 Date'])
    fillna_zip_code(X_train, X_val)
    fillnan_accident_date(X_train,X_val)
    fillnan_birth_year(X_train,X_val)
    feature_creation_has_Cdate (X_train, X_val)
    fill_missing_with_mode(X_train, X_val)
    drop_description_columns(X_train, X_val)
    scaling_standard(X_train, X_val, columns)
    encoding_onehot(X_train, X_val, columns)
    return X_train, X_val
preprocessing_complex(X_train, X_val)

X_train.info()
"""





/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions.py:296: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(mean_value, inplace=True)
/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions.py:297: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work 

<class 'pandas.core.frame.DataFrame'>
Index: 459220 entries, 5785935 to 6027959
Data columns (total 28 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Accident Date                   459220 non-null  float64
 1   Age at Injury                   459220 non-null  float64
 2   Alternative Dispute Resolution  459216 non-null  object 
 3   Assembly Date                   459220 non-null  float64
 4   Attorney/Representative         459220 non-null  object 
 5   Average Weekly Wage             436244 non-null  float64
 6   Birth Year                      459220 non-null  float64
 7   C-2 Date                        459220 non-null  float64
 8   C-3 Date                        149655 non-null  float64
 9   Carrier Name                    459220 non-null  object 
 10  Carrier Type                    459220 non-null  object 
 11  County of Injury                459220 non-null  object 
 12  COVID-19 Indic

# Model Training

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


# Range of k values to test
k_range = range(1, 10)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# To store mean accuracy for each k
mean_accuracies = []

# Perform K-Fold for each k
for k in k_range:
    accuracies = []
    for train_index, test_index in kf.split(X):
        # Split the data into training and test sets for this fold
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the KNN model
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        
        # Evaluate on the test set
        y_pred = knn.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    # Compute the mean accuracy for this k
    mean_accuracies.append(np.mean(accuracies))

# Find the optimal k
optimal_k = k_range[np.argmax(mean_accuracies)]
print(f"The optimal number of neighbors is {optimal_k}.")

# Plot the results
import matplotlib.pyplot as plt
plt.plot(k_range, mean_accuracies)
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Mean Accuracy')
plt.title('Optimal k Selection using K-Fold Cross-Validation')
plt.show()

# Final training with optimal k
final_knn = KNeighborsClassifier(n_neighbors=optimal_k)
final_knn.fit(X, y)
print(f"Model trained with optimal k={optimal_k}.")

  

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Range of k values to test (for XGBoost, we can try different hyperparameters)
learning_rate_range = [0.01, 0.1, 0.3, 0.5, 0.7]
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# To store mean accuracy for each learning rate
mean_accuracies = []

# Perform K-Fold for each learning rate
for learning_rate in learning_rate_range:
    accuracies = []
    for train_index, test_index in kf.split(X):
        # Split the data into training and test sets for this fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the XGBoost model
        model = XGBClassifier(learning_rate=learning_rate, use_label_encoder=False, eval_metric='mlogloss')
        model.fit(X_train, y_train)
        
        # Evaluate on the test set
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    # Compute the mean accuracy for this learning rate
    mean_accuracies.append(np.mean(accuracies))

# Find the optimal learning rate
optimal_lr = learning_rate_range[np.argmax(mean_accuracies)]
print(f"The optimal learning rate is {optimal_lr}.")

# Plot the results
plt.plot(learning_rate_range, mean_accuracies)
plt.xlabel('Learning Rate')
plt.ylabel('Mean Accuracy')
plt.title('Optimal Learning Rate Selection using K-Fold Cross-Validation (XGBoost)')
plt.show()

# Final training with optimal learning rate
final_model = XGBClassifier(learning_rate=optimal_lr, use_label_encoder=False, eval_metric='mlogloss')
final_model.fit(X, y)
print(f"Model trained with optimal learning rate={optimal_lr}.")
